In [ ]:
import pandas as pd
from pandas import DataFrame
import portfolio

In [ ]:
# Read in the pre-processed data
ticker = "AAPL"
data_path = f"../data/{ticker}_processed_hourly_data.csv"
data = pd.read_csv(data_path)

In [ ]:
def calculate_bollinger_bands(df, window=50, num_std=2):
    rolling_mean = df['Close'].rolling(window).mean() 
    rolling_std = df['Close'].rolling(window).std()

    df['Bollinger_Upper'] = rolling_mean + (rolling_std * num_std)
    df['Bollinger_Lower'] = rolling_mean - (rolling_std * num_std)

In [ ]:

calculate_bollinger_bands(data)

# Filter out the rows that will be null
data = data[(data['Bollinger_Upper'].notna()) & (data['Bollinger_Lower'].notna())]

# Set up Portfolio
portfolio = portfolio.Portfolio(initial_cash=100000)
holding = False

In [ ]:
for index, row in data.iterrows():
    if row['Close'] < row['Bollinger_Lower'] and not holding:
        # Buy when price falls below the lower Bollinger Band
        portfolio.buy(row['Close'], 250)
        holding = True
    elif row['Close'] > row['Bollinger_Upper'] and holding:
        # Sell when price rises above the upper Bollinger Band
        portfolio.sell(row['Close'], 250)
        holding = False

print(f"Final Portfolio Value: {portfolio.get_value(data.iloc[-1]['Close'])}")

In [ ]:
display(portfolio.trade_summary())